# Final Project

Analyze the collected Twitter data with OpenAI and store the results in a MongoDB database. The analyses include:


- Extract entities
- Summarize

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install jupyter-ai~=1.0

  Using cached aiosqlite-0.20.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached jupyterlab-3.6.8-py3-none-any.whl.metadata (12 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jupyter_ydoc-0.2.5-py3-none-any.whl.metadata (2.2 kB)
  Using cached jupyter_server_ydoc-0.8.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached nbclassic-1.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached notebook-6.5.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached jupyter_server_fileid-0.9.3-py3-none-any.whl.metadata (4.7 kB)
  Using cached ypy_websocket-0.8.4-py3-none-any.whl.metadata (2.5 kB)
  Using cached y_py-0.6.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached jupyter_client-7.4.9-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl.metadata (755 bytes)
  Using cached aiofiles-22.1.0-py3-none-any.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.9/917.9 kB

In [5]:
pip install jupyter-ai[all]

INFO: pip is looking at multiple versions of langchain-anthropic to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-aws to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-aws to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-cohere to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-mistralai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-nvidia-ai-endpo

In [6]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


## Secret Manager Function

In [8]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Import Python Libraries and Credentials

In [9]:
import pymongo
from pymongo import MongoClient
import json
from pprint import pprint
from tqdm.auto import tqdm
import re

openai_api_key  = get_secret('Openai')['api_key']

mongodb_connect = get_secret('mongodb')['connection_string']

## Connect to the MongoDB cluster

In [10]:
mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
job_collection = db.job_collection #use or create a collection named tweet_collection

## Extract Job Data

Filter the jobs you are interested in. You can use MongoDB Compass to help you write the queries.

In [11]:
filter={

    
}
project={
    'QualificationSummary': 1, 
    'PositionID': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['job_collection'].find(
  filter=filter,
  projection=project
)

Save the extracted jobs into the job_data list.

In [12]:
job_data = []
#url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
for job in result:
   #text_without_urls = re.sub(url_pattern, '', tweet['tweet']['text'])
    job_data.append({'position_id':job['PositionID'],'summary':job['QualificationSummary']})

In [13]:
print('Number of jobs: ',len(job_data))
#The original number of jobs answer I got was 114, I do not understand what has no happened as none of my code has changed.

Number of jobs:  114


## Set up OpenAI API

Load the OpenAI API key and set the API parameters.

- Model type: usegpt-4o by default, and you choose any [availabel models](https://platform.openai.com/docs/models).
- Token estimate: 100 tokens ~= 75 words in English. Total token usage = tokens in the prompt + tokens in the completion. You can get a more accurate estimate at [Tokenier](https://platform.openai.com/tokenizer).
- Temperature: Lower temperatures produce more consistent outputs, while higher values generate more diverse and creative results. 

A help function, ```openai_help```, is created to pass the prompt.

In [11]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)
model="gpt-4o"
temperature=0

def openai_help(prompt, model=model, temperature =temperature ):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

## Sentiment analysis

Analyze the sentiment of each tweet and save the result to the MongoDB database.

In [12]:
for tweet in tqdm(tweet_data):
  
    prompt = f"""
    What is the sentiment of the following tweet, 
    tweet text: {tweet['tweet_text']}
    return  the result with one word as Positive, Neutral,or Negative
 
    """
#     print(prompt)
    try:
        sentiment_result =openai_help(prompt)
    #     print(sentiment_result)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet_id']},
            {"$set":{'tweet.sentiment':sentiment_result}}
        )
    except:
        pass

  0%|          | 0/200 [00:00<?, ?it/s]

## Language translation

Translate each tweet into a different language and save the result to the MongoDB database.

In [13]:
for tweet in tqdm(tweet_data):
  
    prompt = f"""
    Translate the following tweet into Spanish
    tweet text: {tweet['tweet_text']}
 
    """
#     print(prompt)
    try:
        translate_result =openai_help(prompt)
#         print(translate_result)

        tweet_collection.update_one(
            {'tweet.id':tweet['tweet_id']},
            {"$set":{'tweet.translate':translate_result}}
        )
    except:
        pass

  0%|          | 0/200 [00:00<?, ?it/s]

## Identify emotions

Identify whether a tweet expresses anger, and save the result to the MongoDB database.

In [14]:
for tweet in tqdm(tweet_data):
  
    prompt = f"""
    Detect the emotion in the following tweet, and extract whether the tweet expresses anger.
    Provide the result as True, False, or Unknown. 
    Don't provide any reasoning or other output.
    tweet text: {tweet['tweet_text']}
 
    """
#     print(prompt)
    try:
        emotion_result =openai_help(prompt)
        # print(emotion_result)

        tweet_collection.update_one(
                {'tweet.id':tweet['tweet_id']},
                {"$set":{'tweet.anger':emotion_result}}
            )
    except:
        pass

  0%|          | 0/200 [00:00<?, ?it/s]

## Extract Skills

Extract common skills from each job and save the result the the MongoDB database.

In [14]:
for job in tqdm(job_data):
  
    prompt = f"""
    Identify the common technology skills from the following job summary,
    job summary: {job['summary']},
    format the items in a JSON list,
    be consistent, generalize, and concise, if no technology skills presented, use "Unknown" in the list.
    Do not wrap the JSON codes in the JSON markers 
   
    """
#     print(prompt)
    try:
        extract_result =openai_help(prompt)
#        print(extract_result)

        job_collection.update_one(
                {'PositionID':tweet['position_id']},
                {"$set":{'skills':json.loads(extract_result)}}
                )
    except:
        pass

  0%|          | 0/114 [00:00<?, ?it/s]